## Description / Title

Insert description

### Naludaq Version
*Max Version*: `0.17.2`  
*Min Version*: `0.17.2`

In [ ]:
# Print Naludaq version
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

### Compatible Boards
+ `UPAC96`
    + Firmware: `v911` - `v911`

### Equipment Needed
+ Function Generator
+ Oscilloscope

## Example Code

`Give descriptions and/or reasoning of code`

This notebook is to demonstrate how to capture an event with Naludaq's software on the UPAC96 with software trigger.

In [ ]:
%load_ext autoreload
%autoreload 2

from logging import getLogger, DEBUG, StreamHandler, Formatter
import matplotlib.pyplot as plt
import time

from naludaq.board import Board, startup_board
from naludaq.controllers import get_board_controller
from naludaq.daq import DebugDaq

from naluexamples.helpers.plotting import *

### Setup Logger

In [ ]:
def setup_logger(connection_level = DEBUG):
    """Setup a logger for Naludaq to print out debug information.
    
    Args:
        connection_level (int): Sets logging level for UART/FTDI connections.
            Warning: If set too low, VS Code may crash
    """
    logger = getLogger()
    logger.setLevel(DEBUG)

    # UART is very verbose, keep on info unless something is broken.
    # getLogger("naludaq.UART").setLevel(connection_level)
    # getLogger("naludaq.FTDI").setLevel(connection_level)

    handler = StreamHandler()
    handler.setFormatter(Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s"))
    logger.addHandler(handler)

    return logger

try:
    logger
except:
    logger = setup_logger()


### Board Creation and Startup

In [ ]:
MODEL = "upac96"
SERIAL_NUMBER = "FTCKIDF5"
# Refer to BOARD.params["possible_bauds"] keys for a list of available baudrates.
BAUD_RATE = 2000000
BOARD = Board(MODEL)
BOARD.load_registers()
BOARD.get_ftdi_connection(serial_number=SERIAL_NUMBER, baud=BAUD_RATE)
startup_board(BOARD)

### Capture Data

Data is captured with a debugdaq, and events are stored to the DebugDaq's `output_buffer` property

In [ ]:
ddaq = DebugDaq(BOARD)
bc = get_board_controller(BOARD)
# Opening the board with a context manager, ensures the connection is closed when we are done using it
with BOARD as board:
    ddaq.start_capture()
    bc.start_readout("software")
    time.sleep(0.1)
    bc.toggle_trigger()
    time.sleep(3)
    bc.stop_readout()
    ddaq.stop_capture()

### Graph Data

Matplotlib setup

In [ ]:
# Setup graph style
cmap = get_color_mapping("ocean")
set_plot_style(font_size=18, font_family="monospace")

In [ ]:
EVENT = 0
CHANNELS = [0, 1, 2, 3]
DATA = ddaq.output_buffer[EVENT]["data"]
TIME = ddaq.output_buffer[EVENT]["time"]

fig = plt.figure(figsize=(8,8), constrained_layout=True)
plt.xlabel("Time")
plt.ylabel("ADC Counts")
plt.title(f"{MODEL.upper()} Capture")
for channel in CHANNELS:
    ch_data = DATA[channel]
    ch_time = TIME[channel]
    ax = plt.plot(ch_time, ch_data, ".-", label=f"Channel {channel}")
    
plt.legend()
plt.show()